In [ ]:
import networkx as nx
from pyvis.network import Network

from utils import notebook_util, sql_query_util

from db_inference import calc_icity
from IPython.display import display
import pandas as pd

notebook_util.disp_notebook_full_width()


In [ ]:
sql_db = sql_query_util.SimplifiedSqlDb()

In [ ]:
# Interesting combo (high scoring)
# bait_hash = "693388b375dbd274e1"
# tgt_hash = "1b58be4f65b975319f"

bait_hash = "693388b375dbd274e1"
print("BAIT", bait_hash)
tgt_hashes = sql_db.get_p100_windowed_neighbors(bait_hash)
print("TARGETS")
display(tgt_hashes)

tgt_hash = tgt_hashes[0]

In [ ]:
%%time
bait_p30 = sql_db.get_p30_cluster_for_p100(bait_hash)['p30']

tgt_results = []

for tgt_hash in tgt_hashes:
    tgt_p30 = sql_db.get_p30_cluster_for_p100(tgt_hash)['p30']
    tgt_res, bait_res, G = calc_icity.calc_icity_tgt_bait(sql_db, tgt_p30, bait_p30)
    tgt_results.append(tgt_res)
    print(f"calced {tgt_hash}")

icity_df = pd.DataFrame(tgt_results)
icity_df.sort_values('icity_p100', ascending=False, inplace=True)
display(icity_df)

In [ ]:
tgt_p30 = sql_db.get_p30_cluster_for_p100(tgt_hash)['p30']
bait_p30 = sql_db.get_p30_cluster_for_p100(bait_hash)['p30']
tgt_res, bait_res, G = calc_icity.calc_icity_tgt_bait(sql_db, tgt_p30, bait_p30)

In [ ]:
%%time
# Graph connections between target cluster (blue) and bait cluster (purple)
tgt_p30 = sql_db.get_p30_cluster_for_p100(tgt_hash)['p30']
bait_p30 = sql_db.get_p30_cluster_for_p100(bait_hash)['p30']
G = calc_icity.build_icity_graph(sql_db, tgt_p30, bait_p30)

In [ ]:
%%time
net = Network(notebook=True, height="1000px", width="1000px")
net.from_nx(G)
net.show('./pyvis_test.html')